### IBM Data Science Professional Certificate: Capstone Project 
#### <div style="text-align: right"> Author: Tianji Cai </div>   <div style="text-align: right"> Email: tianjiC@outlook.com </div> <div style="text-align: right"> Date: Feb 24th - Mar 4th, 2019 </div> 
# <div style="text-align: center"> *Where is your Second Home?* </div> 
## <div style="text-align: center"> A Recommendation System for Finding Home-like Neighborhoods in a New City</div> 
***

## Introduction

This project aims to build a __recommendation system__ that helps people find the most home-like neighborhoods in a new city. In this notebook, I will use myself as an example and try to look for neighborhoods in __New York and San Francisco__ that are most similar to my hometown in __Shanghai__. Venues around a neighborhood will be obtained from __Foursquare__ and will then be used to construct a feature set which characterizes that particular neighborhood. Afterwards, different neighborhoods in the new cities will be compared to one's hometown based on their respective feature sets, and those that share most similarities with the hometown will be recommended to the customer and get marked on an __interactive Folium map__. From the total number of recommended neighborhoods in a particular city, one can also see which city on the whole most resembles someone's hometown, at least as far as living is concerned. In the case studied here, we will see whether it is New York or San Francisco that is most similar to Shanghai.   

***

## Importing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import geocoder # import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

#!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup # library for web scraping

#!conda install -c conda-forge lxml --yes

import requests # library for requesting webpage

import csv # library for working with csv file

print('Libraries imported.')

Libraries imported.


***

## Table of contents
1. [Background and Business Problem](#introduction)
2. [Data Acquisition](#data)
3. [Methodology and Analysis](#methodology)
4. [Results](#results)
5. [Conclusions and Discussions](#conclusions)

***

## 1. Background and Business Problem <a name="introduction"></a>

### 1.1 Background

As the saying goes, _EAST OR WEST, HOME THE BEST_. After spending years moving from one city to another, I'm often stroke by nostalgia and always wish to find a living place that resembles my childhood home. Given that nowadays globalization has made people much more mobile than ever before, finding a second home in a foreign city that we truly feel comfortable with is likely to become a greater concern for a growing number of young people around the world. Yet, it is no easy task, especially for someone who has to remotely sign an online leasing contract before even visiting the city. 

This is the place where I think data science can help. Utilizing the vast amount of location and venue information available online, if we can somehow build a feature set for every neighborhood in the cities concerned, we can then use these feature sets to compare different neighborhoods with the customer's hometown (or any other neighborhood he/she picks) and come up with the top neighborhoods that most resemble one's hometown. The result can aid our customers to make a well-informed choice in deciding where to live. 

As an analogy, the recommendation system that will be built here is similar in essence to that used by Netflix which suggests the next movie for you to watch.  

### 1.2 Business Problem

As said in the _Introduction_, I will use myself as an example for this project. As a child, I lived in a district called __Hongkou__ in __Shanghai, China__, which used to be a cultural center and is located not far from the commercial heart of Shanghai. It is a nice place to live and I will use the neighborhoods in this district/borough as a benchmark in my search for a second home far far away on the other side of the globe. 

Since I'm an aspiring data scientist, the next city that I might move into will probably be a hub for data science. I decided to pick one city from the east coast of US and one from the west coast, hoping that this will enable me to make a crude comparison of the two coasts in addition to my neighborhood analysis. The two cities chosen in this study are __New York and San Francisco__. __The main objective is to recommend a total number of 20 neighborhoods in these two cities__. From the recommendation, we can also learn which city has the largest number of recommended neighborhoods, thus most similar to my hometown in Shanghai. 

Choosing your right neighborhood is a daunting task. Therefore, the analysis done here will be interesting to anyone who plans to move into a new city, yet is at a loss in the process of finding a new place to call home.

***

## 2. Data Acquisition <a name="data"></a>

### 2.1 Description of Data

Based on the business problem defined above, we need two sets of data:
* Neighborhoods in a given city with latitudes and longitudes
* Venues information in a given neighborhood

#### 2.1.1 Neighborhood Location Data

For the 1st set, i.e, location data of the neighborhoods, we need to obtain tables that have the following form:

| City | Borough | Neighborhood | Latitude | Longitude |
| --- | --- | --- | --- | --- |
| --- | --- | --- | --- | --- |

We need 3 such tables, each for one city being considered in the project. In addition, we will create a master table _TwoCities_ for the two US cities combined, and a grand master dataframe _ThreeCities_ for all three cities combined.

* __Shanghai__: I'm only interested in neighborhoods in the district/borough of Hongkou and they can be found on this [Wikipedia page](https://en.wikipedia.org/wiki/Hongkou_District). Since there're only 8 neighborhoods and I'm not able to find the latitudes and longitudes of these neighborhoods all in one table, I decided to create the dataframe myself and put in the values by hand. The brute-force method is more time-saving than scraping the web in this case, so I will go with it. Below is how the table should look like, where all the location data are obtained from direct Google search.

| City | Borough | Neighborhood | Latitude | Longitude |
| --- | --- | --- | --- | --- |
| Shanghai | Hongkou | Ouyang | 31.2695 | 121.4943 |
| Shanghai | Hongkou | Quyang | 31.2890 | 121.4953 |
| Shanghai | Hongkou | Guangzhong | 31.2669 | 121.4775 |
| Shanghai | Hongkou | Jiaxing | 31.2714 | 121.5002 |
| Shanghai | Hongkou | Liangcheng Xincun| 31.2988 | 121.4719 |
| Shanghai | Hongkou | Sichuan North | 31.2584 | 121.4806 |
| Shanghai | Hongkou | Tilanqiao | 31.2564 | 121.5137 |
| Shanghai | Hongkou | Jiangwanzhen | 31.3049 | 121.4732 |


* __New York__: We've already created such a dataframe in our example project of exploring neighborhoods in New York, and there the raw data was fetched from this [json file](https://cocl.us/new_york_dataset). I'll simply repeat the procedures in that project, which makes life much easier.

* __San Francisco__: It is not easy to find an existing dataset for the neighborhoods in San Francisco and after much work I did find one from [DataSF.org](https://data.sfgov.org/Geographic-Locations-and-Boundaries/Realtor-Neighborhoods/5gzd-g9ns). The data downloaded from their csv/json file is not very organized and I need to clean them before the anaylsis. In addition, since the data was originally collected in 2010, it might not be most up-to-date and by comparing it with the [Wikipedia Page](https://en.wikipedia.org/wiki/List_of_neighborhoods_in_San_Francisco), I did find some discrepancies. However, given that I'm in no way concerned with precision, the dataset here should suffice our purpose in this project. 

#### 2.1.2 Venue Information Data

It's relatively easy to obtain this data. We will use __Foursquare__ API to fetch the data, same as what we did before. We will create two dataframes of venue categories, one for the neighborhoods in Hongkou and the other for those in New York and San Francisco. 

In order to get the same number of venue categories for each city, we need to fetch the venue data for all three cities at the same time. Thus, we will use our grand master _ThreeCities_ dataframe here. Afterwards, the venue categories dataframe will be split into two, and the one for Shanghai will be averaged to get a _HomeFeatures_ dataframe, which defines the characteristics of my hometown and will be used in the same way as a user profile. The other dataframe, named _NewNeighborFeatures_, gives us the venue characteristics of each neighborhood in NY and SF.  

The _HomeFeatures_ dataframe will then be multiplied by the _NewNeighborFeatures_ dataframe to produce a sorted _NewNeighborRating_ dataframe, whose top few rows tell us the most home-like neighborhoods in NY and SF.  

**The final result will be the first 20 rows of _NewNeighborRating_ dataframe, given in a new and argumented dataframe _RecommendedNeighborhoods_.**

### 2.2 Cleaning Neighborhood Location Data

#### 2.2.1 Shanghai

Let's build the dataframe for _Hongkou, Shanghai_ by hand.

In [2]:
# define the dataframe columns
column_names = ['City', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
HomeLocation = pd.DataFrame(columns=column_names)
HomeLocation

,City,Borough,Neighborhood,Latitude,Longitude


In [3]:
HomeNeigh_name = ['Ouyang', 'Quyang', 'Guangzhong', 'Jiaxing', 'Liangcheng Xincun', 'Sichuan North', 'Tilanqiao', 'Jiangwanzhen']
HomeNeigh_lat = [31.2695, 31.2890, 31.2669, 31.2714, 31.2988, 31.2584, 31.2564, 31.3049]
HomeNeigh_lon = [121.4943, 121.4953, 121.4775, 121.5002, 121.4719, 121.4806, 121.5137, 121.4732]

for i in range(len(HomeNeigh_name)):
    HomeLocation = HomeLocation.append({'City': 'Shanghai',
                                    'Borough': 'Hongkou',
                                    'Neighborhood': HomeNeigh_name[i],
                                    'Latitude': HomeNeigh_lat[i],
                                    'Longitude': HomeNeigh_lon[i]}, ignore_index=True)
HomeLocation

,City,Borough,Neighborhood,Latitude,Longitude
0,Shanghai,Hongkou,Ouyang,31.2695,121.4943
1,Shanghai,Hongkou,Quyang,31.2890,121.4953
2,Shanghai,Hongkou,Guangzhong,31.2669,121.4775
3,Shanghai,Hongkou,Jiaxing,31.2714,121.5002
4,Shanghai,Hongkou,Liangcheng Xincun,31.2988,121.4719
5,Shanghai,Hongkou,Sichuan North,31.2584,121.4806
6,Shanghai,Hongkou,Tilanqiao,31.2564,121.5137
7,Shanghai,Hongkou,Jiangwanzhen,31.3049,121.4732


We now have __HomeLocation__ as our dataframe for the location data for neighborhoods in Hongkou, Shanghai.

Let's visualize the neighborhoods near my home on a folium map.

In [4]:
# Get the latitude and longitude of Hongkou, Shanghai, using geopy library.
address = 'Hongkou, Shanghai, CN'

geolocator = Nominatim(user_agent="Shanghai_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Hongkou, Shanghai are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Hongkou, Shanghai are 31.266703, 121.501751.


In [10]:
# create map of Shanghai using latitude and longitude values
map_hkshanghai = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(HomeLocation['Latitude'], HomeLocation['Longitude'], HomeLocation['Borough'], HomeLocation['Neighborhood']):
    label = '{}: {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_hkshanghai)  
    
map_hkshanghai

I have to apologize that the characters written on the map are in Chinese. But at least you can see the blue spots, which are all that matter.

#### 2.2.2 New York

Let's build the dataframe for _New York_, using the same procedure as in the project Neighborhoods-New-York.

In [6]:
url = 'https://cocl.us/new_york_dataset'
newyork_data = requests.get(url).json()
NY_data = newyork_data['features']
NY_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [7]:
# define the dataframe columns
column_names = ['City', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
NYLocation = pd.DataFrame(columns=column_names)
NYLocation

,City,Borough,Neighborhood,Latitude,Longitude


In [8]:
for data in NY_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NYLocation = NYLocation.append({'City': 'New York',
                                          'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

NYLocation.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939
2,New York,Bronx,Eastchester,40.887556,-73.827806
3,New York,Bronx,Fieldston,40.895437,-73.905643
4,New York,Bronx,Riverdale,40.890834,-73.912585


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(NYLocation['Borough'].unique()),
        NYLocation.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


We now have __NYLocation__ as our dataframe for the location data for neighborhoods in New York, US.

Let's also visualize the neighborhoods in New York on a folium map.

In [11]:
# Get the latitude and longitude of New York, using geopy library.
address = 'New York City, NY'

geolocator = Nominatim(user_agent="newyork_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(NYLocation['Latitude'], NYLocation['Longitude'], NYLocation['Borough'], NYLocation['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Oh God, as you can see on the map, there're a lot of neighborhoods in New York!

#### 2.2.3 San Francisco

Let's read in the local csv file for the neighborhoods data in San Francisco and take a quick look at it.

In [13]:
SFLocation = pd.read_csv("SF_Neighborhoods.csv")
SFLocation.head()

,sfar_distr,the_geom,nbrhood,nid
0,District 6 - Central North,MULTIPOLYGON (((-122.42948394891741 37.7750962...,Alamo Square,6e
1,District 6 - Central North,MULTIPOLYGON (((-122.44746439135872 37.7798633...,Anza Vista,6a
2,District 4 - Twin Peaks West,MULTIPOLYGON (((-122.46450886214802 37.7322084...,Balboa Terrace,4a
3,District 10 - Southeast,MULTIPOLYGON (((-122.38758527038996 37.7502633...,Bayview,10a
4,District 9 - Central East,MULTIPOLYGON (((-122.40375492236231 37.7491900...,Bernal Heights,9a


We need the name of neighborhoods and boroughs from the file, as well as its location.

In [14]:
# Separate the location data from the df
location = SFLocation['the_geom']

# Renaming the two useful columns
SFLocation.rename(columns ={'sfar_distr': 'Borough', 'nbrhood': 'Neighborhood'}, inplace=True) 
# Dropping the other columns
SFLocation.drop(columns=['the_geom', 'nid'], inplace=True)
# Deleting the characters "District # - " in the names of boroughs
for i in range(11):
    SFLocation['Borough'].replace(regex=True,inplace=True,to_replace=r'District {} - '.format(i),value=r'')
# Retaining only one neighborhood if several are present in one cell    
SFLocation['Neighborhood'] = SFLocation['Neighborhood'].apply(lambda x: x.split('/')[0])
# Add the other three empty columns 
SFLocation = SFLocation.reindex(['City'] + SFLocation.columns.tolist() + ['Latitude', 'Longitude'], axis=1) 
# Set all the values of 'City' to 'San Francisco'
SFLocation['City'] = 'San Francisco'

# Inspect the data
rows = SFLocation.shape[0]
print('The number of rows of SFLocation is: ', rows)
SFLocation.head()

The number of rows of SFLocation is:  92


,City,Borough,Neighborhood,Latitude,Longitude
0,San Francisco,Central North,Alamo Square,NaN,NaN
1,San Francisco,Central North,Anza Vista,NaN,NaN
2,San Francisco,Twin Peaks West,Balboa Terrace,NaN,NaN
3,San Francisco,Southeast,Bayview,NaN,NaN
4,San Francisco,Central East,Bernal Heights,NaN,NaN


Now clean the location data and append them back into SFLocation

In [15]:
for row in range(rows):
    location[row] = location[row].replace("MULTIPOLYGON (((","")
    location[row] = location[row][:location[row].find(",")]
    latitude = location[row].split(" ")[1]
    longitude = location[row].split(" ")[0]
    
    SFLocation['Latitude'][row] = latitude
    SFLocation['Longitude'][row] = longitude
    
SFLocation.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,City,Borough,Neighborhood,Latitude,Longitude
0,San Francisco,Central North,Alamo Square,37.775096,-122.429484
1,San Francisco,Central North,Anza Vista,37.779863,-122.447464
2,San Francisco,Twin Peaks West,Balboa Terrace,37.732208,-122.464509
3,San Francisco,Southeast,Bayview,37.750263,-122.387585
4,San Francisco,Central East,Bernal Heights,37.749190,-122.403755


In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(SFLocation['Borough'].unique()),
        SFLocation.shape[0]
    )
)

The dataframe has 11 boroughs and 92 neighborhoods.


We now have __SFLocation__ as our dataframe for the location data for neighborhoods in San Francisco, US.

Let's finally visualize the neighborhoods in San Francisco on a folium map.

In [16]:
# Get the latitude and longitude of San Francisco, using geopy library.
address = 'San Francisco, CA, USA'

geolocator = Nominatim(user_agent="sanfrancisco_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Francisco are 37.7792808, -122.4192363.


In [17]:
# create map of New York using latitude and longitude values
map_sanfrancisco = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(SFLocation['Latitude'], SFLocation['Longitude'], SFLocation['Borough'], SFLocation['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sanfrancisco)  
    
map_sanfrancisco

#### _HomeLocation_, _NYLocation_, _SFLocation_ are the three location dataframe that will be used in the subsequent analysis!

#### 2.2.4 Master Dataframes _TwoCities_ and _ThreeCities_

Let's merge _NYLocation_ and _SFLocation_ into one dataframe _TwoCities_.

In [18]:
TwoCities = NYLocation.append(SFLocation, ignore_index=True)
print('The shape of TwoCities is: ', TwoCities.shape)
TwoCities

The shape of TwoCities is:  (398, 5)


,City,Borough,Neighborhood,Latitude,Longitude
0,New York,Bronx,Wakefield,40.894705,-73.847201
1,New York,Bronx,Co-op City,40.874294,-73.829939
2,New York,Bronx,Eastchester,40.887556,-73.827806
3,New York,Bronx,Fieldston,40.895437,-73.905643
4,New York,Bronx,Riverdale,40.890834,-73.912585
5,New York,Bronx,Kingsbridge,40.881687,-73.902818
6,New York,Manhattan,Marble Hill,40.876551,-73.910660
7,New York,Bronx,Woodlawn,40.898273,-73.867315
8,New York,Bronx,Norwood,40.877224,-73.879391
9,New York,Bronx,Williamsbridge,40.881039,-73.857446


Let's merge _HomeLocation_ and _TwoCities_ into one dataframe _ThreeCities_.

In [19]:
ThreeCities = HomeLocation.append(TwoCities, ignore_index=True)
print('The shape of ThreeCities is: ', ThreeCities.shape)
ThreeCities

The shape of ThreeCities is:  (406, 5)


,City,Borough,Neighborhood,Latitude,Longitude
0,Shanghai,Hongkou,Ouyang,31.269500,121.494300
1,Shanghai,Hongkou,Quyang,31.289000,121.495300
2,Shanghai,Hongkou,Guangzhong,31.266900,121.477500
3,Shanghai,Hongkou,Jiaxing,31.271400,121.500200
4,Shanghai,Hongkou,Liangcheng Xincun,31.298800,121.471900
5,Shanghai,Hongkou,Sichuan North,31.258400,121.480600
6,Shanghai,Hongkou,Tilanqiao,31.256400,121.513700
7,Shanghai,Hongkou,Jiangwanzhen,31.304900,121.473200
8,New York,Bronx,Wakefield,40.894705,-73.847201
9,New York,Bronx,Co-op City,40.874294,-73.829939


### Note: 

Here in our record, New York has a total of 306 neighborhoods, while San Francisco only has 92 neighborhoods. Therefore statistically speaking, New York should have more neighborhoods (__roughly 3 times more__) that are similar to my home than San Francisco, if those home-like neighborhoods are assumed to be randomly distribued among the two cities. This should be kept in mind when we later perform our analysis.

***

## 3. Methodology and Analysis <a name="methodology"></a>

In this section, we will carry out our main analysis of the project. The procedure can be broken down into the following steps:
* Obtaining Venue Information Data 
* Cleaning Venue Information Data
* Building the User Profile: _HomeFeatures_ dataframe
* Characteristics of Neighborhoods in NY and SF: _NewNeighborFeatures_ dataframe
* Providing Recommendation: _NewNeighborRating_ dataframe

The dataframes mentioned above are already discussed in section 2.1.2, and more explanations will be provided along with the actual analysis.

### 3.1 Obtaining Venue Information Data 

First using the dataframe _ThreeCities_, let's get the top 100 venues within a radius of 1000 meters of each neighborhood from __Foursquare API__.

In [20]:
# Define Foursquare credentials and version
CLIENT_ID = '2SAZNGTUB5AHNAGD02E2IPT0UI0BULJI1HRQRA1HVZFPOGY4' 
CLIENT_SECRET = 'UNN4SDP4WABDM0CEX2IDBJMFBM2HX55MYTRMJ2ZQEUFAWPVS' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2SAZNGTUB5AHNAGD02E2IPT0UI0BULJI1HRQRA1HVZFPOGY4
CLIENT_SECRET:UNN4SDP4WABDM0CEX2IDBJMFBM2HX55MYTRMJ2ZQEUFAWPVS


In [21]:
# Define a function to get relevant information about venues in a neighborhhod.
def getNearbyVenues(city, borough, neighborhood, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for city, borough, neighborhood, lat, lng in zip(city, borough, neighborhood, latitudes, longitudes):
        print(city, borough, neighborhood)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            city, 
            borough, 
            neighborhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City',
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# Get the venues for all the neighborhoods in the three cities
venues = getNearbyVenues(city=ThreeCities['City'],
                         borough=ThreeCities['Borough'],
                         neighborhood=ThreeCities['Neighborhood'],
                         latitudes=ThreeCities['Latitude'],
                         longitudes=ThreeCities['Longitude'])

Shanghai Hongkou Ouyang
Shanghai Hongkou Quyang
Shanghai Hongkou Guangzhong
Shanghai Hongkou Jiaxing
Shanghai Hongkou Liangcheng Xincun
Shanghai Hongkou Sichuan North
Shanghai Hongkou Tilanqiao
Shanghai Hongkou Jiangwanzhen
New York Bronx Wakefield
New York Bronx Co-op City
New York Bronx Eastchester
New York Bronx Fieldston
New York Bronx Riverdale
New York Bronx Kingsbridge
New York Manhattan Marble Hill
New York Bronx Woodlawn
New York Bronx Norwood
New York Bronx Williamsbridge
New York Bronx Baychester
New York Bronx Pelham Parkway
New York Bronx City Island
New York Bronx Bedford Park
New York Bronx University Heights
New York Bronx Morris Heights
New York Bronx Fordham
New York Bronx East Tremont
New York Bronx West Farms
New York Bronx High  Bridge
New York Bronx Melrose
New York Bronx Mott Haven
New York Bronx Port Morris
New York Bronx Longwood
New York Bronx Hunts Point
New York Bronx Morrisania
New York Bronx Soundview
New York Bronx Clason Point
New York Bronx Throgs Neck


New York Queens Jamaica Hills
New York Queens Utopia
New York Queens Pomonok
New York Queens Astoria Heights
New York Bronx Claremont Village
New York Bronx Concourse Village
New York Bronx Mount Eden
New York Bronx Mount Hope
New York Manhattan Sutton Place
New York Queens Hunters Point
New York Manhattan Turtle Bay
New York Manhattan Tudor City
New York Manhattan Stuyvesant Town
New York Manhattan Flatiron
New York Queens Sunnyside Gardens
New York Queens Blissville
New York Brooklyn Fulton Ferry
New York Brooklyn Vinegar Hill
New York Brooklyn Weeksville
New York Brooklyn Broadway Junction
New York Brooklyn Dumbo
New York Staten Island Manor Heights
New York Staten Island Willowbrook
New York Staten Island Sandy Ground
New York Staten Island Egbertville
New York Queens Roxbury
New York Brooklyn Homecrest
New York Queens Middle Village
New York Staten Island Prince's Bay
New York Staten Island Lighthouse Hill
New York Staten Island Richmond Valley
New York Queens Malba
New York Brook

Let's explore the venues in each neighborhood returned by Foursquare.

In [23]:
# Inspect our dataframe venues
print('The shape of venues is: ', venues.shape)
venues.head(10)

The shape of venues is:  (13773, 9)


,City,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Shanghai,Hongkou,Ouyang,31.2695,121.4943,Ruihong Tiandi (瑞虹天地月亮湾),31.266713,121.494406,Shopping Mall
1,Shanghai,Hongkou,Ouyang,31.2695,121.4943,Starbucks (星巴克),31.265544,121.492827,Coffee Shop
2,Shanghai,Hongkou,Ouyang,31.2695,121.4943,Street Food,31.271859,121.492699,BBQ Joint
3,Shanghai,Hongkou,Ouyang,31.2695,121.4943,Youdian Xincun Metro Station (邮电新村地铁站),31.270422,121.489823,Metro Station
4,Shanghai,Hongkou,Quyang,31.2890,121.4953,同济大学音乐广场,31.285461,121.495471,Student Center
5,Shanghai,Hongkou,Quyang,31.2890,121.4953,上海教育超市,31.285226,121.495512,Food & Drink Shop
6,Shanghai,Hongkou,Quyang,31.2890,121.4953,艾尼巴亿餐厅 ‖Aini Bayi,31.288421,121.490652,Chinese Restaurant
7,Shanghai,Hongkou,Quyang,31.2890,121.4953,同济大学樱花大道,31.284725,121.496804,Park
8,Shanghai,Hongkou,Guangzhong,31.2669,121.4775,多伦路文化名人街,31.265602,121.477287,Art Gallery
9,Shanghai,Hongkou,Guangzhong,31.2669,121.4775,万寿斋,31.267822,121.480192,Chinese Restaurant


In [24]:
# Find out how many unique categories can be curated from all the returned venues
print('There are {} uniques venue categories.'.format(len(venues['Venue Category'].unique())))

There are 480 uniques venue categories.


In [25]:
# Check how many venues were returned for each neighborhood
print("There're {} neighborhoods returned by Foursquare.".format(venues.groupby(['City', 'Borough', 'Neighborhood']).count().shape[0]))
venues.groupby(['City', 'Borough', 'Neighborhood']).count()

There're 405 neighborhoods returned by Foursquare.


Neighborhood Latitude  \
City          Borough         Neighborhood                                       
New York      Bronx           Allerton                                      25   
                              Baychester                                    24   
                              Bedford Park                                  37   
                              Belmont                                       96   
                              Bronxdale                                     15   
                              Castle Hill                                    8   
                              City Island                                   24   
                              Claremont Village                             18   
                              Clason Point                                  11   
                              Co-op City                                    14   
                              Concourse                                     25   
                              Concourse Village                             35   
                              Country Club                                   8   
                              East Tremont                                  17   
                              Eastchester                                   19   
                              Edenwald                                       6   
                              Edgewater Park                                22   
                              Fieldston                                      4   
                              Fordham                                       89   
                              High  Bridge                                  26   
                              Hunts Point                                   11   
                              Kingsbridge                                   73   
                              Kingsbridge Heights                           40   
                              Longwood                                      11   
                              Melrose                                       25   
                              Morris Heights                                 9   
                              Morris Park                                   24   
                              Morrisania                                    22   
                              Mott Haven                                    22   
                              Mount Eden                                    32   
                              Mount Hope                                     9   
                              North Riverdale                               22   
                              Norwood                                       28   
                              Olinville                                     12   
                              Parkchester                                   39   
                              Pelham Bay                                    39   
                              Pelham Gardens                                19   
                              Pelham Parkway                                26   
                              Port Morris                                   16   
                              Riverdale                                      8   
                              Schuylerville                                 20   
                              Soundview                                     16   
                              Spuyten Duyvil                                 6   
                              Throgs Neck                                   11   
                              Unionport                                     20   
                              University Heights                            26   
                              Van Nest                                      14   
                              Wakefield

We've lost 406 - 405 = 1 neighborhood! We will find it out later. 

#### Now, all the venue information data for all neighborhoods of the three cities are stored in the data frame _venues_.

### 3.2 Cleaning Venue Information Data 

Now let's clean the venue data and obtain the useful information that gives the number of venues in each category for every neighborhood.

In [26]:
# one hot encoding
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add city, borough, neighborhood columns back to dataframe
venues_onehot['City'] = venues['City'] 
venues_onehot['Borough'] = venues['Borough'] 
venues_onehot['Neighborhood'] = venues['Neighborhood'] 

# move city, borough, neighborhood columns to the first three columns
cols = venues_onehot.columns.tolist()
cols.insert(0, cols.pop(cols.index('City')))
cols.insert(1, cols.pop(cols.index('Borough')))
cols.insert(2, cols.pop(cols.index('Neighborhood')))
venues_onehot = venues_onehot.reindex(columns=cols)

print('The shape of venues_onehot is: ', venues_onehot.shape)
venues_onehot.head()

The shape of venues_onehot is:  (13773, 482)


,City,Borough,Neighborhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Misce

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [27]:
venues_grouped = venues_onehot.groupby(['City', 'Borough', 'Neighborhood']).mean().reset_index()
print('The shape of venues_grouped is ', venues_grouped.shape)
venues_grouped.head()

The shape of venues_grouped is  (405, 482)


,City,Borough,Neighborhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Misce

Let's now find out which neighborhood is lost.

In [30]:
ThreeCities[~ThreeCities['Neighborhood'].isin(venues_grouped['Neighborhood'])]

,City,Borough,Neighborhood,Latitude,Longitude
401,San Francisco,Central East,Central Waterfront,37.750941,-122.375964


So Central WaterFront in Central East San Francisco is lost. Never mind, let's just delete it from our analysis.

#### The dataframe *venues_grouped* contains all the venue information that we need for subsequent analysis.

### 3.3 Building the User Profile: _HomeFeatures_ dataframe

First, get the venue categories information for neighborhoods near my home from *venues_grouped* and store them in *HomeVenues*.

In [38]:
HomeVenues = venues_grouped[venues_grouped['Neighborhood'].isin(HomeLocation['Neighborhood'])].reset_index(drop=True)
print('The HomeVenues has shape: ', HomeVenues.shape)
HomeVenues

The HomeVenues has shape:  (8, 482)


,City,Borough,Neighborhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Misce

Now average across neighborhoods to get a new dataframe *HomeFeatures* that characterises the neighborhoods around my home.

In [39]:
HomeFeatures = HomeVenues.drop(['City','Borough','Neighborhood'], 1).mean()
HomeFeatures

Acai House                                  0.000000
Accessories Store                           0.000000
Adult Boutique                              0.000000
Afghan Restaurant                           0.000000
African Restaurant                          0.000000
Airport Terminal                            0.000000
Airport Tram                                0.000000
Alternative Healer                          0.000000
American Restaurant                         0.000000
Amphitheater                                0.000000
Animal Shelter                              0.000000
Antique Shop                                0.000000
Aquarium                                    0.000000
Arcade                                      0.000000
Arepa Restaurant                            0.000000
Argentinian Restaurant                      0.020833
Art Gallery                                 0.025000
Art Museum                                  0.000000
Arts & Crafts Store                         0.

#### We now have *HomeFeatures* to be used as the user profile.

### 3.4 Characteristics of Neighborhoods in NY and SF: _NewNeighborFeatures_ dataframe

Obtain the venue categories information for neighborhoods in NY and SF from *venues_grouped* and store them in *NewNeighborVenues*.

In [40]:
NewNeighborVenues = venues_grouped[venues_grouped['Neighborhood'].isin(TwoCities['Neighborhood'])].reset_index(drop=True)
print('The NewNeighborVenues has shape: ', NewNeighborVenues.shape)
NewNeighborVenues.head()

The NewNeighborVenues has shape:  (397, 482)


,City,Borough,Neighborhood,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Misce

Drop *City*, *Borough*, and *Neighborhood* columns to obtain the dataframe *NewNeighborFeatures*. 

In [41]:
NewNeighborFeatures = NewNeighborVenues.drop(['City','Borough','Neighborhood'], 1)
NewNeighborFeatures.head()

,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,Airport Tram,Alternative Healer,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dive Shop,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Medical School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone

#### We now have *NewNeighborFeatures*.

### 3.5 Providing Recommendation: _NewNeighborRating_ dataframe

With the user profile *HomeFeatures* and the complete list of neighborhoods and their venue categories *NewNeighborFeatures* at hand, we're going to multiply *NewNeighborFeatures* by *HomeFeatures* and take the weighted average across rows to get an overall similarity score for each neighborhood in NY and SF. After sorting, this will create our final result dataframe *NewNeighborRating* and enable us to recommend the top twenty neighborhoods that most resemble my home. 

In [42]:
#Multiply NewNeighborFeatures by HomeFeatures and then take the weighted average
recommendation = (NewNeighborFeatures * HomeFeatures).sum(axis=1)/(HomeFeatures.sum())
print('The shape of recommendation is: ', recommendation.shape)
recommendation.head()

The shape of recommendation is:  (397,)


0    0.008022
1    0.004706
2    0.009655
3    0.010451
4    0.018309
dtype: float64

In [43]:
# Append the names of cities, boroughs and neighborhoods to recommendation
NewNeighborRating = pd.concat([NewNeighborVenues['City'], NewNeighborVenues['Borough'], NewNeighborVenues['Neighborhood'], recommendation], axis=1, sort=False)
NewNeighborRating.rename(columns={NewNeighborRating.columns[3]:'Similarity Score'}, inplace=True) 
NewNeighborRating.sort_values(by=['Similarity Score'], ascending=False, inplace=True)
NewNeighborRating.reset_index(drop=True, inplace=True)
print('The shape of NewNeighborRating is:', NewNeighborRating.shape)
NewNeighborRating.head()

The shape of NewNeighborRating is: (397, 4)


,City,Borough,Neighborhood,Similarity Score
0,New York,Queens,Somerville,0.124185
1,New York,Staten Island,Todt Hill,0.062092
2,New York,Bronx,Clason Point,0.047052
3,San Francisco,Northwest,Lone Mountain,0.043946
4,San Francisco,None,Golden Gate Park,0.043946


#### We now obtained our result: _NewNeighborRating_!
***

## 4. Results <a name="results"></a>

Now the first 20 items in *NewNeighborRating* are the neighborhoods most similar to my home. Let's store them in a new dataframe: _RecommendedNeighborhoods_.

In [122]:
RecommendedNeighborhoods = NewNeighborRating.head(20)
RecommendedNeighborhoods 

,City,Borough,Neighborhood,Similarity Score
0,New York,Queens,Somerville,0.124185
1,New York,Staten Island,Todt Hill,0.062092
2,New York,Bronx,Clason Point,0.047052
3,San Francisco,Northwest,Lone Mountain,0.043946
4,San Francisco,None,Golden Gate Park,0.043946
5,New York,Bronx,Spuyten Duyvil,0.041395
6,San Francisco,Twin Peaks West,Sherwood Forest,0.041395
7,San Francisco,Twin Peaks West,Monterey Heights,0.041395
8,New York,Queens,Bayswater,0.041395
9,San Francisco,Twin Peaks West,Westwood Highlands,0.035481


Let's add latitudes and longitudes for these neighborhoods. 

In [123]:
# Add two additional columns to RecommendedNeighborhoods
Place = TwoCities[TwoCities['Neighborhood'].isin(RecommendedNeighborhoods['Neighborhood'])]
RecommendedNeighborhoods = RecommendedNeighborhoods.merge(Place, on=['City', 'Borough', 'Neighborhood'], how='left')
RecommendedNeighborhoods

,City,Borough,Neighborhood,Similarity Score,Latitude,Longitude
0,New York,Queens,Somerville,0.124185,40.597711,-73.796648
1,New York,Staten Island,Todt Hill,0.062092,40.597069,-74.111329
2,New York,Bronx,Clason Point,0.047052,40.806551,-73.854144
3,San Francisco,Northwest,Lone Mountain,0.043946,37.774655,-122.454745
4,San Francisco,None,Golden Gate Park,0.043946,37.774655,-122.454745
5,New York,Bronx,Spuyten Duyvil,0.041395,40.881395,-73.917190
6,San Francisco,Twin Peaks West,Sherwood Forest,0.041395,37.736208,-122.459155
7,San Francisco,Twin Peaks West,Monterey Heights,0.041395,37.730721,-122.458216
8,New York,Queens,Bayswater,0.041395,40.611322,-73.765968
9,San Francisco,Twin Peaks West,Westwood Highlands,0.035481,37.736597,-122.453722


Let's visualize these neighborhoods on two folium maps, one centered around NY, and the other around SF.

In [64]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="newyork_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(RecommendedNeighborhoods['Latitude'], RecommendedNeighborhoods['Longitude'], RecommendedNeighborhoods['Borough'], RecommendedNeighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [137]:
address = 'San Francisco, CA, USA'

geolocator = Nominatim(user_agent="sanfrancisco_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_sanfrancisco = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(RecommendedNeighborhoods['Latitude'], RecommendedNeighborhoods['Longitude'], RecommendedNeighborhoods['Borough'], RecommendedNeighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sanfrancisco)  
    
map_sanfrancisco

Let's count how many of the recommended neighborhoods are in NY or SF.

In [59]:
RecommendedNeighborhoods.groupby('City').count()

,Borough,Neighborhood,Similarity Score,Latitude,Longitude
City,,,,,
New York,10,10,10,10,10
San Francisco,10,10,10,10,10


#### There're 10 home-like neighborhoods in New York, and 10 in San Francisco. That is a ratio of 1:1. 
#### But since New York has a total of 3 times more neighborhoods than San Francisco, we should get roughly a ratio of 3:1 if home-like neighborhoods are equally distributed among the two cities. Thus, we conclude that San Francisco is more like my home in Shanghai than New York. 
***

## 5. Conclusions and Discussions <a name="conclusions"></a>

Again, we've found 10 neighborhoods in New York and San Francisco respectively that are most similar to my home in Hongkou, Shanghai. These 20 most home-like neighborhoods, together with their similarity score, is given in __RecommendedNeighborhoods__ (see below). 

In [121]:
RecommendedNeighborhoods

,City,Borough,Neighborhood,Similarity Score,Latitude,Longitude
0,New York,Queens,Somerville,0.124185,40.597711,-73.796648
1,New York,Staten Island,Todt Hill,0.062092,40.597069,-74.111329
2,New York,Bronx,Clason Point,0.047052,40.806551,-73.854144
3,San Francisco,Northwest,Lone Mountain,0.043946,37.774655,-122.454745
4,San Francisco,None,Golden Gate Park,0.043946,37.774655,-122.454745
5,New York,Bronx,Spuyten Duyvil,0.041395,40.881395,-73.917190
6,San Francisco,Twin Peaks West,Sherwood Forest,0.041395,37.736208,-122.459155
7,San Francisco,Twin Peaks West,Monterey Heights,0.041395,37.730721,-122.458216
8,New York,Queens,Bayswater,0.041395,40.611322,-73.765968
9,San Francisco,Twin Peaks West,Westwood Highlands,0.035481,37.736597,-122.453722


Since New York have more neighborhoods to begin with, we conclude that __San Francisco__ is more similar to Shanghai, as far as living is concerned. 

From the table, we can see that the top one neighborhoods has a same similarity score which is substantially higher than the rest. We thus recommend __Somerville, Queens, NY__ as the most home-like neighborhood. 

Finally, let's further find out the 5 most common venues in Somerville, Queens, NY, and Lone Mountain, Northwest, SF, which are the top home-like neighborhoods in their respective cities.

In [126]:
# First define a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [135]:
# Select the subset of venues_grouped that contain the 2 neighborhoods
venues_somerville = venues_grouped[venues_grouped['Neighborhood']=='Somerville'].reset_index(drop=True)
venues_lonemountain = venues_grouped[venues_grouped['Neighborhood']=='Lone Mountain'].reset_index(drop=True)

# Create the new dataframe
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create new dataframes
venues_sorted_somerville = pd.DataFrame(columns=columns)
venues_sorted_somerville['Neighborhood'] = venues_somerville['Neighborhood']
venues_sorted_lonemountain = pd.DataFrame(columns=columns)
venues_sorted_lonemountain['Neighborhood'] = venues_lonemountain['Neighborhood']

for ind in np.arange(venues_somerville.shape[0]):
    venues_sorted_somerville.iloc[ind, 1:] = return_most_common_venues(venues_somerville.iloc[ind, :], num_top_venues)
    venues_sorted_lonemountain.iloc[ind, 1:] = return_most_common_venues(venues_lonemountain.iloc[ind, :], num_top_venues)


venues_sorted_somerville

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Somerville,Park,Zoo,Food Court,Farmers Market,Fast Food Restaurant


In [136]:
venues_sorted_lonemountain

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Lone Mountain,Park,Café,Deli / Bodega,Middle Eastern Restaurant,Garden


### Discussions:

In the future, the analysis done here can be expanded to include more cities. One __general commercial application__ might be a more comprehensive recommendation system that include major cities around the world. In order to accomplish this goal, we will need to maintain our own database that stores all the location data for neighborhoods in different cities. We will also need a commercial account on Foursquare so that we can fetch unlimited amount of venue data at our will. In doing this project, I did encounter the difficulty set by an upper limit of daily calls of Foursquare API, which makes it impossible to fetch all the venue data more than once daily. 

Another caveat is that Foursquare does NOT always return the same venue information. The first time I ran this project, no neighborhood was lost and I got 481 different venue categories. But the second time I ran exactly the same code again, I only got 480 unique venue categories and one neighborhood was lost. Of course, this affects the subsequent analysis, and the resulting 20 recommended neighborhoods are therefore different for run 1 and run 2. This notebook, together with the report and the blog post, shows the result for run 2. 

***

#### This concludes my current project.  Hope you like it. Thanks!

#### Author: Tianji Cai